In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.

# Let's make a quick call to a Frontier model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to have your first message here. How can I assist you today?


## OK onwards with our first project

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

trt = Website("https://www.trthaber.com")
print(trt.title)
print(trt.text)

TRT Haber: Son Dakika Haberler ile Türkiye’nin Haber Kaynağı
SICAK
GÜNDEM
TÜRKİYE
DÜNYA
TÜRK DÜNYASI
EKONOMİ
SPOR
SAVUNMA
ÇOCUK
ÖZEL HABER
DOSYA HABER
DİĞER
SON HABERLER
TÜM MANŞETLER
FOTO FOKUS
DÜNYA DIŞI
VİDEO GALERİ
KÜLTÜR-SANAT
DOSYA HABER
YAŞAM
SAĞLIK
GEZİ
TEKNOLOJİ
ÇEVRE
EĞİTİM
GÜNCEL
HAVA DURUMU
PODCAST
HAVA UYARILARI
TRT'DEN HABERLER
TRT ARŞİV
TRT AKADEMİ
PROGRAMLAR
11ºC
Ankara
Adana
Adıyaman
Afyonkarahisar
Ağrı
Aksaray
Amasya
Ankara
Antalya
Ardahan
Artvin
Aydın
Balıkesir
Bartın
Batman
Bayburt
Bilecik
Bingöl
Bitlis
Bolu
Burdur
Bursa
Çanakkale
Çankırı
Çorum
Denizli
Diyarbakır
Düzce
Edirne
Elazığ
Erzincan
Erzurum
Eskişehir
Gaziantep
Giresun
Gümüşhane
Hakkari
Hatay
Iğdır
Isparta
İstanbul
İzmir
Kahramanmaraş
Karabük
Karaman
Kars
Kastamonu
Kayseri
Kırıkkale
Kırklareli
Kırşehir
Kilis
Kocaeli
Konya
Kütahya
Malatya
Manisa
Mardin
Mersin
Muğla
Muş
Nevşehir
Niğde
Ordu
Osmaniye
Rize
Sakarya
Samsun
Siirt
Sinop
Sivas
Şanlıurfa
Şırnak
Tekirdağ
Tokat
Trabzon
Tunceli
Uşak
Van
Yalova
Yozgat
Zong

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
system_prompt

'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'

In [10]:
print(user_prompt_for(trt))

You are looking at a website titled TRT Haber: Son Dakika Haberler ile Türkiye’nin Haber Kaynağı
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

SICAK
GÜNDEM
TÜRKİYE
DÜNYA
TÜRK DÜNYASI
EKONOMİ
SPOR
SAVUNMA
ÇOCUK
ÖZEL HABER
DOSYA HABER
DİĞER
SON HABERLER
TÜM MANŞETLER
FOTO FOKUS
DÜNYA DIŞI
VİDEO GALERİ
KÜLTÜR-SANAT
DOSYA HABER
YAŞAM
SAĞLIK
GEZİ
TEKNOLOJİ
ÇEVRE
EĞİTİM
GÜNCEL
HAVA DURUMU
PODCAST
HAVA UYARILARI
TRT'DEN HABERLER
TRT ARŞİV
TRT AKADEMİ
PROGRAMLAR
11ºC
Ankara
Adana
Adıyaman
Afyonkarahisar
Ağrı
Aksaray
Amasya
Ankara
Antalya
Ardahan
Artvin
Aydın
Balıkesir
Bartın
Batman
Bayburt
Bilecik
Bingöl
Bitlis
Bolu
Burdur
Bursa
Çanakkale
Çankırı
Çorum
Denizli
Diyarbakır
Düzce
Edirne
Elazığ
Erzincan
Erzurum
Eskişehir
Gaziantep
Giresun
Gümüşhane
Hakkari
Hatay
Iğdır
Isparta
İstanbul
İzmir
Kahramanmaraş
Karabük
Karaman
Kars
Kastamonu
Kayseri
Kırıkkale
Kırklareli
Kırşehir
Kili

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [11]:
#messages = [
#    {"role": "system", "content": "You are a snarky assistant"},
#    {"role": "user", "content": "What is 2 + 2?"}
#]

In [12]:
# To give you a preview -- calling OpenAI with system and user messages:

#response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
#print(response.choices[0].message.content)

## And now let's build useful messages for GPT-4o-mini, using a function

In [13]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
# Try this out, and then try for a few more websites

messages_for(trt)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled TRT Haber: Son Dakika Haberler ile Türkiye’nin Haber Kaynağı\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSICAK\nGÜNDEM\nTÜRKİYE\nDÜNYA\nTÜRK DÜNYASI\nEKONOMİ\nSPOR\nSAVUNMA\nÇOCUK\nÖZEL HABER\nDOSYA HABER\nDİĞER\nSON HABERLER\nTÜM MANŞETLER\nFOTO FOKUS\nDÜNYA DIŞI\nVİDEO GALERİ\nKÜLTÜR-SANAT\nDOSYA HABER\nYAŞAM\nSAĞLIK\nGEZİ\nTEKNOLOJİ\nÇEVRE\nEĞİTİM\nGÜNCEL\nHAVA DURUMU\nPODCAST\nHAVA UYARILARI\nTRT\'DEN HABERLER\nTRT ARŞİV\nTRT AKADEMİ\nPROGRAMLAR\n11ºC\nAnkara\nAdana\nAdıyaman\nAfyonkarahisar\nAğrı\nAksaray\nAmasya\nAnkara\nAntalya\nArdahan\nArtvin\nAydın\nBalıkesir\nBartın\nBatman\nBayburt\nBilec

## Time to bring it together - the API for OpenAI is very simple!

In [15]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://www.trthaber.com")

# TRT Haber: Son Dakika Haberler ile Türkiye’nin Haber Kaynağı

TRT Haber, Türkiye'nin güncel olayları ve dünyanın dört bir yanındaki gelişmeleri içeren haberleri sunan bir platformdur. Web sitesinde geniş bir yelpazede haber kategorileri bulunmaktadır; bunlar arasında gündem, ekonomi, spor, kültür-sanat, sağlık ve çevre gibi konular yer alır. Ayrıca, sıcak haberlere ve detaylı dosya haberlere erişim sağlamak mümkündür.

## Son Haberler Özeti:
- **Cumhurbaşkanı Erdoğan, Mattia Ahmet Minguzzi'nin ailesiyle görüştü:** Erdoğan, İstanbul'da öldürülen Minguzzi'nin ailesine taziye ziyaretinde bulundu.
- **CHP'nin boykot çağrısına ilişkin 11 kişi gözaltına alındı:** Ekonomiye zarar vermek amacıyla yapılan boykot çağrısına karşılık bazı kişiler gözaltına alındı.
- **Polonya Savunma Bakanı, AB'nin NATO'nun yerini almak istemediğini belirtti:** NATO ve Avrupa Birliği ilişkilerine dair açıklamalar yapıldı.
- **Türkiye-AB ekonomik diyalogu toplantısı:** Türkiye ve Avrupa Birliği arasında ekonomik işbirliği görüşmeleri planlanıyor.
- **Virajı alamayan bir tır, park halindeki araçların üzerine devrildi:** Türkiye'de trafik kazalarıyla ilgili son gelişmeler.

Site ayrıca hava durumu, günlük döviz kurları ve genel yaşam ile ilgili içeriklere de yer vermektedir. Kullanıcılar, bu platform üzerinden ülke ve dünya genelindeki gelişmeleri takip edebilirler.